In [ ]:
#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## GPUによる高速化

TensorflowやPyTorchといった深層学習ライブラリはGPUでの実行がサポートされています。
GPUを使うことで大規模計算が高速化できることがあります。

このノートブックではTensorflow/KerasやPyTorchをGPU上で実行し、CPUでの実行したときと計算速度を比較します。


### GPUの利用状況の確認

現在接続しているノードのGPU使用率は以下のコマンドで確認できます。
```bash
$ nvidia-smi
Thu Jul 20 11:29:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  On   | 00000000:1F:00.0 Off |                    0 |
| N/A   84C    P0   129W / 250W |   3493MiB / 40960MiB |     78%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PCI...  On   | 00000000:20:00.0 Off |                    0 |
| N/A   84C    P0   175W / 250W |   2823MiB / 40960MiB |     79%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+

+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
|        ID   ID                                                   Usage      |
|=============================================================================|
|    0   N/A  N/A   3460006      C   python                           1242MiB |
|    0   N/A  N/A   3461170      C   ...HostWorker.run_executable     1494MiB |
|    1   N/A  N/A   3460006      C   python                            598MiB |
|    1   N/A  N/A   3461170      C   ...HostWorker.run_executable     2216MiB |
+-----------------------------------------------------------------------------+
```
このノードにはGPUが2台搭載されており、それぞれ2つのプロセスがGPUを使用しています。

自分のプロセスがゾンビ状態となってGPUのメモリを専有してしまうことがないように定期的にチェックするのが好ましいです。

GPUを使っているプロセスを起動したユーザーを確認するコマンドの一例は以下です。
```bash
$ ps -up `nvidia-smi --query-compute-apps=pid --format=csv,noheader | sort -u`
USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
```
ここで表示されるメモリの値はGPU上のメモリではないことに注意してください。

notebook上では"!"をつけることでbashコマンドが実行できます。     

In [ ]:
!nvidia-smi

In [ ]:
!ps -up `nvidia-smi --query-compute-apps=pid --format=csv,noheader | sort -u`

### 使用するGPUの指定
まずは使うGPUを指定します。このステップなしでもGPUは利用できますが、GPUを専有してしまうことで共有マシンを使用している他のユーザーに迷惑をかけてしまうことがあります。
今回の講習では、使うGPUは1つのみに限定させます。

In [ ]:
def get_available_gpu_index():
    import random
    import subprocess

    # ノード上のGPUのインデックスとuuidを取得します。
    p = subprocess.run(
        f"nvidia-smi --query-gpu=index,gpu_uuid --format=csv,noheader",
        shell=True,
        stdout=subprocess.PIPE,
    )
    gpu_uuid_to_index = {}
    for v in p.stdout.decode().split('\n'):
        if len(v) == 0:
            continue
        index, uuid = v.split(',')
        gpu_uuid_to_index[uuid.strip()] = index.strip()

    # GPUを使っているプロセスのuuidを取得します。
    p = subprocess.run(
        f"nvidia-smi --query-compute-apps=gpu_uuid --format=csv,noheader",
        shell=True,
        stdout=subprocess.PIPE,
    )
    ret = p.stdout.decode().split('\n')
    ret = set(ret)
    for uuid in ret:
        if len(uuid) == 0:
            continue
        gpu_uuid_to_index.pop(uuid)

    # どのプロセスからも使用されていないGPUのインデックスをランダムに一つ返します。
    # GPUが使用できない場合は空文字列を返します。
    if len(gpu_uuid_to_index) == 0:
        print('GPU is unavailable.')
        return ""
    else:
        available_gpu_index = list(gpu_uuid_to_index.values())
        return random.choice(available_gpu_index)

# 使用されていないGPUを選択します。
gpu_index = get_available_gpu_index()
print(gpu_index)

# 使うGPUを一つに制限します。
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_index



In [ ]:
# PyTorchが使うCPUの数を制限します。(VMを使う場合)
import os
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

from torch import set_num_threads, set_num_interop_threads
num_threads = 1
set_num_threads(num_threads)
set_num_interop_threads(num_threads)

### GPU上でのPyTorchの計算
PyTorchでは入力データ(tensor)やモデルをどのデバイスで計算させるか明示的に指定する必要があります。
tensorやモデルオブジェクトに`.cuda()`、または`.to('cuda')`を指定することでGPUメモリにオブジェクトをコピーできます。

#### tensor オブジェクトのGPUメモリへの移動
```python
from torch import tensor
tensor(1)  # CPU
tensor(1).cpu()  # CPU
tensor(1).cuda()  # GPU
tensor(1).to('cpu')  # CPU
tensor(1).to('cuda')  # GPU
```

#### Model オブジェクトのGPUメモリへの移動
```python
from torch.nn import Sequential
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import ReLU
model = Sequential(
    Linear(in_features=2, out_features=32),
    Sigmoid(),
)

model  # CPU
model.cpu()  # CPU
model.cuda()  # GPU
model.cpu()  # CPU
model.to('cpu')  # CPU
model.to('cuda')  # GPU
```

## MLPのトレーニング

MLPのトレーニングをCPU/GPUでそれぞれ実行することで、計算時間の変化を確認します。

データセットは乱数で適当に作成します。

In [ ]:
from numpy.random import default_rng
rng = default_rng(seed=0)

# 10万行、入力次元10のランダムなデータを作成。
datasize = 100000
x = rng.normal(size=(datasize, 10))
t = rng.integers(0, 2, size=(datasize, 1))

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
from torch import tensor
ds = TensorDataset(tensor(x).float(), tensor(t).float())
dataloader = DataLoader(ds, batch_size=1024, shuffle=True)

## CPUによるサンプルコード

PyTorchでは明示的に指定しない限り、CPU上で計算が行われます。

In [ ]:
from torch.nn import Sequential
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import ReLU
from torch.nn import BCELoss
from torch.optim import Adam
from torch import from_numpy
from torch.cuda import synchronize
import time

# モデルの定義
model = Sequential(
    Linear(in_features=10, out_features=256),  # ノード数が256の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=256, out_features=256),  # ノード数が256の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=256, out_features=256),  # ノード数が256の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=256, out_features=256),  # ノード数が256の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=256, out_features=256),  # ノード数が256の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=256, out_features=1),  # ノード数が1の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
)
# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = BCELoss()
optimizer = Adam(model.parameters())

# トレーニング
for i_epoch in range(10):

    # エポックごとのロス、accuracyを計算するための変数
    loss_total = 0.

    # 時間の測定
    synchronize()
    start_time = time.time()

    for x, t in dataloader:
        # 順伝搬
        y_pred = model(x)

        # ロスの計算
        loss = loss_fn(y_pred, t)
        loss_total += loss.detach().numpy()

        # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
        # これをしないと、勾配の値はそれまでの値との和がとられる。
        optimizer.zero_grad()

        # 誤差逆伝播。各パラメータの勾配が計算される。
        loss.backward()

        # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
        optimizer.step()

    # ロス、accuracyをミニバッチの数で割って平均を取ります。
    loss_total /= len(dataloader)
    synchronize()  # GPUの処理が終わるのを待ちます。
    print(f'epoch = {i_epoch}, time = {time.time() - start_time: .3f} sec, loss = {loss_total}')



## GPUによるサンプルコード
tensorやmodelをGPU上に移動する(`.cuda()`の追加)ことで、GPUを使ったトレーニングができます。

In [ ]:
from torch.nn import Sequential
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import ReLU
from torch.nn import BCELoss
from torch.optim import Adam
from torch import from_numpy
from torch.cuda import synchronize
import time

# モデルの定義
model = Sequential(
    Linear(in_features=10, out_features=256),  # ノード数が256の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=256, out_features=256),  # ノード数が256の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=256, out_features=256),  # ノード数が256の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=256, out_features=256),  # ノード数が256の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=256, out_features=256),  # ノード数が256の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=256, out_features=1),  # ノード数が1の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
)
# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = BCELoss()
optimizer = Adam(model.parameters())

# GPUメモリにモデルをコピーします。
model = model.cuda()

# トレーニング
for i_epoch in range(10):

    # エポックごとのロス、accuracyを計算するための変数
    loss_total = 0.

    # 時間の測定
    synchronize()
    start_time = time.time()

    for x, t in dataloader:
        # GPUメモリにデータをコピーします。
        x = x.cuda()
        t = t.cuda()

        # 順伝搬
        y_pred = model(x)

        # ロスの計算
        loss = loss_fn(y_pred, t)
        loss_total += loss.detach().cpu().numpy()  # lossの合計値を記録するため、値をCPUにコピーします。

        # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
        # これをしないと、勾配の値はそれまでの値との和がとられる。
        optimizer.zero_grad()

        # 誤差逆伝播。各パラメータの勾配が計算される。
        loss.backward()

        # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
        optimizer.step()

    # ロス、accuracyをミニバッチの数で割って平均を取ります。
    loss_total /= len(dataloader)
    synchronize()  # GPUの処理が終わるのを待ちます。
    print(f'epoch = {i_epoch}, time = {time.time() - start_time: .3f} sec, loss = {loss_total}')



計算時間はどのように変化したでしょうか。
モデルサイズが変化すると、計算時間はどのように変わるでしょうか？